In [2]:
import xarray as xr
from glob import glob
import pandas as pd
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import thread_map
import os
wave_wind_mapping = {
    # Combined Wave Parameters
    "Hsig": "HTSGW",       # Hsig: Significant wave height -> HTSGW: Significant Height of Combined Wind Waves and Swell
    "Dir": "DIRPW",      # PkDir: Peak wave direction -> DIRPW: Direction of Combined Wind Waves and Swell (Often peak direction is reported as the primary direction)
    "Tm01": "PERPW",       # Tm01: Mean wave period (m01) -> PERPW: Mean Period of Combined Wind Waves and Swell (Tm01 is a common representation for mean period)
    # Wind Parameters
    #"Windv_x": "UGRD",     # Windv_x: Horizontal component of wind velocity -> UGRD: U-component of wind
    #"Windv_y": "VGRD",     # Windv_y: Vertical component of wind velocity (Likely intended as the other horizontal component) -> VGRD: V-component of wind
}
files = pd.Series(sorted(glob("hcast_nc/*.nc")))
files

0       hcast_nc/NI-000101_000630-DEPTH.nc
1         hcast_nc/NI-000101_000630-DIR.nc
2        hcast_nc/NI-000101_000630-DSPR.nc
3        hcast_nc/NI-000101_000630-FSPR.nc
4       hcast_nc/NI-000101_000630-HSIGN.nc
                       ...                
2157      hcast_nc/SI-990701_991231-RTP.nc
2158     hcast_nc/SI-990701_991231-TM02.nc
2159      hcast_nc/SI-990701_991231-TPS.nc
2160     hcast_nc/SI-990701_991231-WIND.nc
2161     hcast_nc/SI-990701_991231-WLEN.nc
Length: 2162, dtype: object

In [2]:
os.makedirs("Joao", exist_ok=True)
for island in ["SI", "NI"]:
  for year in tqdm(range(2000, 2020)):
    year_chop = year - 2000
    dsets = xr.open_mfdataset(files[files.str.contains(f"(?:{island})-(?:{year_chop:02d}).+-(?:HSIGN|DIR|RTM01)")].reset_index(drop=True), combine="nested")
    dsets.to_netcdf(f"Joao/{island}-{year}.nc", mode="w")

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [4]:
ds = xr.open_mfdataset("Joao/NI-*.nc", decode_times=False)
display(ds)
ds.to_netcdf("Joao/NI.nc", mode="w")

<xarray.Dataset> Size: 14GB
Dimensions:    (time: 58440, latitude: 103, longitude: 100)
Coordinates:
  * time       (time) float64 468kB 1.753e+05 1.753e+05 ... 3.506e+05 3.506e+05
  * latitude   (latitude) float32 412B -42.99 -42.9 -42.81 ... -33.56 -33.46
  * longitude  (longitude) float32 400B 170.4 170.5 170.6 ... 179.4 179.5 179.6
Data variables:
    Dir        (time, latitude, longitude) float64 5GB dask.array<chunksize=(2928, 103, 100), meta=np.ndarray>
    Hsig       (time, latitude, longitude) float64 5GB dask.array<chunksize=(2928, 103, 100), meta=np.ndarray>
    RTm01      (time, latitude, longitude) float64 5GB dask.array<chunksize=(2928, 103, 100), meta=np.ndarray>

In [5]:
ds = xr.open_mfdataset("Joao/SI-*.nc", decode_times=False)
display(ds)
ds.to_netcdf("Joao/SI.nc", mode="w")

<xarray.Dataset> Size: 17GB
Dimensions:    (time: 58440, latitude: 103, longitude: 121)
Coordinates:
  * time       (time) float64 468kB 1.753e+05 1.753e+05 ... 3.506e+05 3.506e+05
  * latitude   (latitude) float32 412B -48.04 -47.94 -47.85 ... -38.6 -38.51
  * longitude  (longitude) float32 484B 165.6 165.7 165.8 ... 176.6 176.7 176.8
Data variables:
    Dir        (time, latitude, longitude) float64 6GB dask.array<chunksize=(2928, 103, 121), meta=np.ndarray>
    Hsig       (time, latitude, longitude) float64 6GB dask.array<chunksize=(2928, 103, 121), meta=np.ndarray>
    RTm01      (time, latitude, longitude) float64 6GB dask.array<chunksize=(2928, 103, 121), meta=np.ndarray>